### Env Config

In [1]:
%%capture
%pip install pandas requests fastparquet

In [2]:
import zipfile, requests, os, random
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

In [3]:
def convert_category_dtype(df: pd.DataFrame, column: str):
    df[column] = df[column].astype('category')
    return df
def convert_float_dtype(df: pd.DataFrame, column: str):
    df[column] = df[column].apply(lambda x : str(x).replace(",","."))
    df[column] = df[column].astype('float64')
    return df
def convert_int_dtype(df: pd.DataFrame, column: str):
    df[column] = df[column].apply(lambda x : str(x).replace("<= 15","15"))
    df[column] = df[column].astype(int)
    return df

def export_to_parquet(df: pd.DataFrame, path: str):
	if not os.path.exists(path):
		os.makedirs(path)
	df.to_parquet(f'{path}/df.parquet', engine='fastparquet')

### Extração Manual e manutenção da integridade da zona Staging

In [4]:
planilhas = [
	'planilha_2012',
	'planilha_2013',
	'planilha_2014',
	'planilha_2015',
	'planilha_2016',
	'planilha_2017',
	'planilha_2018',
	'planilha_2019',
	'planilha_2020',
	'planilha_2021',
	'planilha_2022',
	'planilha_2023',
	'planilha_2024',
]

for planilha in planilhas:
	if not os.path.exists(f'data/staging/{planilha}.zip'):
		url = f'https://www.bcb.gov.br/pda/desig/{planilha}.zip'

		print(f'Baixando arquivo "{planilha}.zip"...')
		response = requests.get(url)

		with open(f'data/staging/{planilha}.zip', 'wb') as f:
			f.write(response.content)

### Exportando para zona Raw

In [5]:
if not os.path.exists('data/raw'):
	os.makedirs('data/raw')

for planilha in planilhas:
    zip_obj = zipfile.ZipFile(f'data/staging/{planilha}.zip', 'r')
    extracted_files = zip_obj.namelist()

    for file in extracted_files:
        extracted_path = os.path.join(f'data/raw', file)
        
        if not os.path.exists(extracted_path):
            print(f'Extraindo {file}...')
            try:
                zip_obj.extract(file, 'data/raw')
            except zipfile.BadZipFile:
                print(f"Erro: O arquivo {planilha}.zip está corrompido.")
            except Exception as e:
                print(f"Erro ao extrair {file}: {e}")

    zip_obj.close()

### Criando dataframe

In [6]:
files = os.listdir('data/raw')
files.sort()

total_lines_size = 2000000
# Essa opção de randomização existe pois o dataset é ordenado por UF, e isso tanto pode enviesar a análise quanto remove a opção de utilizar UF como uma variável categórica válida
randomize_sample = True

sample_size = total_lines_size / len(files)
sample_size = int(sample_size)

df = pd.DataFrame()
for file in files:
	print(f'Processado o arquivo {file}')

	temp_df = pd.DataFrame()
	if randomize_sample:
		temp_df = pd.read_csv(f'data/raw/{file}', sep=';', encoding='utf-8-sig').sample(n=sample_size, random_state=random.randint(1, 1000))
	else:
		temp_df = pd.read_csv(f'data/raw/{file}', sep=';', encoding='utf-8-sig', nrows=sample_size)
	df = pd.concat([df, temp_df], ignore_index=True)

Processado o arquivo planilha_201206.csv
Processado o arquivo planilha_201207.csv
Processado o arquivo planilha_201208.csv
Processado o arquivo planilha_201209.csv
Processado o arquivo planilha_201210.csv
Processado o arquivo planilha_201211.csv
Processado o arquivo planilha_201212.csv
Processado o arquivo planilha_201301.csv
Processado o arquivo planilha_201302.csv
Processado o arquivo planilha_201303.csv
Processado o arquivo planilha_201304.csv
Processado o arquivo planilha_201305.csv
Processado o arquivo planilha_201306.csv
Processado o arquivo planilha_201307.csv
Processado o arquivo planilha_201308.csv
Processado o arquivo planilha_201309.csv
Processado o arquivo planilha_201310.csv
Processado o arquivo planilha_201311.csv
Processado o arquivo planilha_201312.csv
Processado o arquivo planilha_201401.csv
Processado o arquivo planilha_201402.csv
Processado o arquivo planilha_201403.csv
Processado o arquivo planilha_201404.csv
Processado o arquivo planilha_201405.csv
Processado o arq

### Ajuste de tipagem dos dados

In [7]:
df.dtypes

data_base                         object
uf                                object
tcb                               object
sr                                object
cliente                           object
ocupacao                          object
cnae_secao                        object
cnae_subclasse                    object
porte                             object
modalidade                        object
origem                            object
indexador                         object
numero_de_operacoes               object
a_vencer_ate_90_dias              object
a_vencer_de_91_ate_360_dias       object
a_vencer_de_361_ate_1080_dias     object
a_vencer_de_1081_ate_1800_dias    object
a_vencer_de_1801_ate_5400_dias    object
a_vencer_acima_de_5400_dias       object
vencido_acima_de_15_dias          object
carteira_ativa                    object
carteira_inadimplida_arrastada    object
ativo_problematico                object
dtype: object

In [8]:
FLOAT_COLS = [
    "a_vencer_ate_90_dias",
    "a_vencer_de_91_ate_360_dias",
    "a_vencer_de_361_ate_1080_dias",
    "a_vencer_de_1081_ate_1800_dias",
    "a_vencer_de_1801_ate_5400_dias",
    "a_vencer_acima_de_5400_dias",
    "vencido_acima_de_15_dias",
    "carteira_ativa",
    "carteira_inadimplida_arrastada",
    "ativo_problematico"
]
CATEGORY_COLS = [
    'uf',
    'tcb',
    'sr',
    'ocupacao',
    'cnae_secao',
    'cnae_subclasse',
    'porte',
    'modalidade',
    'origem',
    'indexador'
]
INT_COLS = [
    'numero_de_operacoes'
]

for column in df.columns:
	if column in FLOAT_COLS:
		df = convert_float_dtype(df=df, column=column)
		df = df.rename(columns={column: "vl_" + column.lower()})
	elif column in CATEGORY_COLS:
		df = convert_category_dtype(df=df, column=column)
		df = df.rename(columns={column: "ct_" + column.lower()})
	elif column in INT_COLS:
		df = convert_int_dtype(df=df, column=column)
		df = df.rename(columns={column: "nu_" + column.lower()})

In [9]:
df['dt_data_base'] = pd.to_datetime(df.pop('data_base'), format="%Y-%m-%d")

In [10]:
df['dt_data_base'].dtype.name

'datetime64[ns]'

In [11]:
df['cliente'].value_counts()

cliente
PJ    1883304
PF     306696
Name: count, dtype: int64

In [12]:
df['ct_classificacao'] = df.pop('cliente').astype('category')

#### Assim percebemos que já existe uma coluna com a classificação do cliente, então vamos retirar essa informação das outras colunas

In [13]:
df['ct_porte'].value_counts()

ct_porte
PJ - Pequeno                                     684083
PJ - Micro                                       675847
PJ - Médio                                       431197
PJ - Grande                                       82023
PF - Mais de 5 a 10 salários mínimos              43156
PF - Mais de 3 a 5 salários mínimos               41486
PF - Mais de 10 a 20 salários mínimos             40037
PF - Acima de 20 salários mínimos                 39734
PF - Mais de 2 a 3 salários mínimos               39014
PF - Mais de 1 a 2 salários mínimos               38718
PF - Até 1 salário mínimo                         32457
PF - Sem rendimento                               20400
PF - Indisponível                                 11694
PJ - Indisponível                                 10154
Name: count, dtype: int64

In [14]:
df['ct_porte'] = df['ct_porte'].apply(lambda x: x[5:].strip()).astype('category')

In [15]:
df['ct_modalidade'].value_counts()

ct_modalidade
PJ - Capital de giro                                                              429691
PJ - Outros créditos                                                              360866
PJ - Cheque especial e conta garantida                                            306160
PJ - Investimento                                                                 302158
PJ - Financiamento de infraestrutura/desenvolvimento/projeto e outros créditos    265904
PJ - Operações com recebíveis                                                     169486
PF - Outros créditos                                                               76950
PF - Empréstimo sem consignação em folha                                           48012
PF - Cartão de crédito                                                             42549
PF - Rural e agroindustrial                                                        40011
PF - Veículos                                                                      35660
PF - Em

In [16]:
df['ct_modalidade'] = df['ct_modalidade'].apply(lambda x: x[5:].strip()).astype('category')

In [17]:
df['ct_ocupacao'].value_counts()

ct_ocupacao
-                                                  1883304
PF - Outros                                          54026
PF - Empresário                                      46386
PF - Autônomo                                        44241
PF - Servidor ou empregado público                   43613
PF - Empregado de empresa privada                    43204
PF - Aposentado/pensionista                          38627
PF - MEI                                             20481
PF - Empregado de entidades sem fins lucrativos      16118
Name: count, dtype: int64

In [18]:
df['ct_ocupacao'] = df['ct_ocupacao'].apply(lambda x: x[5:].strip()).astype('category')

In [19]:
df['ct_cnae_secao'].value_counts()

ct_cnae_secao
PJ - Comércio; reparação de veículos automotores e motocicletas           634330
PJ - Indústrias de transformação                                          419309
-                                                                         306696
PJ - Atividades administrativas e serviços complementares                 115023
PJ - Construção                                                           100093
PJ - Transporte, armazenagem e correio                                     90060
PJ - Atividades profissionais, científicas e técnicas                      79347
PJ - Saúde humana e serviços sociais                                       73835
PJ - Informação e comunicação                                              61048
PJ - Alojamento e alimentação                                              49233
PJ - Outras atividades de serviços                                         48942
PJ - Educação                                                              48742
PJ - Agricultu

In [20]:
df['ct_cnae_secao'] = df['ct_cnae_secao'].apply(lambda x: x[5:].strip()).astype('category')

In [21]:
df['ct_cnae_subclasse'].value_counts()

ct_cnae_subclasse
-                                                                                                                                    373365
PJ - Transporte rodoviário de carga, exceto produtos perigosos e mudanças, intermunicipal, interestadual e internacional              10378
PJ - Construção de edifícios                                                                                                           9786
PJ - Comércio varejista de materiais de construção em geral                                                                            9390
PJ - Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - minimercados, mercearias e armazéns      9339
                                                                                                                                      ...  
PJ - Fundos de investimento previdenciários                                                                                               1
PJ

In [22]:
df['ct_cnae_subclasse'] = df['ct_cnae_subclasse'].apply(lambda x: x[5:].strip()).astype('category')

### Exportando para zona Trusted

In [23]:
export_to_parquet(df, 'data/trusted')

In [24]:
df['vl_carteira_ativa_n_arrastada'] = df['vl_carteira_ativa'] - df['vl_carteira_inadimplida_arrastada']

In [25]:
df['vl_carteira_ativa_n_arrastada'].describe()

count       2190000.00
mean        5486553.27
std       105398384.07
min               0.00
25%           16222.25
50%           95875.54
75%          530343.21
max     23078512568.43
Name: vl_carteira_ativa_n_arrastada, dtype: float64

In [26]:
print((df['vl_a_vencer_acima_de_5400_dias'] != 0.00).sum())
print((df['vl_a_vencer_de_1801_ate_5400_dias'] != 0.00).sum())
print((df['vl_a_vencer_de_361_ate_1080_dias'] != 0.00).sum())
print((df['vl_a_vencer_de_91_ate_360_dias'] != 0.00).sum())
print((df['vl_a_vencer_de_1081_ate_1800_dias'] != 0.00).sum())
print((df['vl_a_vencer_ate_90_dias'] != 0.00).sum())

37150
313744
1357788
1814698
780684
2000874


In [27]:
CONVERSAO_FAIXAS = {
	'vl_a_vencer_acima_de_5400_dias': '> 180',
 	'vl_a_vencer_de_1801_ate_5400_dias': '36-180',
	'vl_a_vencer_de_1081_ate_1800_dias': '18-36',
	'vl_a_vencer_de_361_ate_1080_dias': '12-18',
	'vl_a_vencer_de_91_ate_360_dias': '3-12',
	'vl_a_vencer_ate_90_dias': '0-3',
}

def calcula_range(row):
    for col, val in CONVERSAO_FAIXAS.items():
        if row[col] != 0.00:
            return val
    return None

df['ct_faixa_meses_ate_vencimento'] = df.apply(calcula_range, axis=1).astype('category')

In [28]:
df['ct_faixa_meses_ate_vencimento'].value_counts()

ct_faixa_meses_ate_vencimento
12-18     579947
18-36     471288
3-12      471174
36-180    279493
0-3       272512
> 180      37150
Name: count, dtype: int64

In [29]:
df['vl_media_carteira_ativa_por_operacao'] = df['vl_carteira_ativa'] / df['nu_numero_de_operacoes']

In [30]:
df['vl_media_carteira_ativa_por_operacao'].describe()

count     2190000.00
mean        95563.54
std       2734528.20
min             0.00
25%          1085.79
50%          4656.33
75%         19184.75
max     909839085.40
Name: vl_media_carteira_ativa_por_operacao, dtype: float64

In [31]:
df['vl_media_carteira_inadimplida_por_operacao'] = df['vl_carteira_inadimplida_arrastada'] / df['nu_numero_de_operacoes']

In [32]:
df['vl_media_carteira_inadimplida_por_operacao'].describe()

count     2190000.00
mean         2073.38
std        216571.46
min             0.00
25%             0.00
50%             0.00
75%            72.62
max     240960862.62
Name: vl_media_carteira_inadimplida_por_operacao, dtype: float64

In [33]:
df.dtypes

ct_uf                                               category
ct_tcb                                              category
ct_sr                                               category
ct_ocupacao                                         category
ct_cnae_secao                                       category
ct_cnae_subclasse                                   category
ct_porte                                            category
ct_modalidade                                       category
ct_origem                                           category
ct_indexador                                        category
nu_numero_de_operacoes                                 int64
vl_a_vencer_ate_90_dias                              float64
vl_a_vencer_de_91_ate_360_dias                       float64
vl_a_vencer_de_361_ate_1080_dias                     float64
vl_a_vencer_de_1081_ate_1800_dias                    float64
vl_a_vencer_de_1801_ate_5400_dias                    float64
vl_a_vencer_acima_de_540

In [34]:
df['ct_uf'].value_counts()

ct_uf
SP    242360
RS    198013
PR    181427
SC    180618
MG    169356
RJ    111636
GO    102613
BA     92645
MT     87501
ES     81985
PE     71955
MS     68893
PA     64670
CE     62905
DF     58204
MA     46523
RO     45199
RN     43848
AM     43517
PB     43293
TO     36701
AL     36144
PI     33791
SE     33554
AC     19364
AP     17851
RR     15434
Name: count, dtype: int64

### Exportando para zona Refined

In [35]:
export_to_parquet(df, 'data/refined')
df.to_csv(f'data/refined/df.csv', index=False)